#Import statements and global variables assignement

In [1]:
import cv2
import numpy as np
import random,os,subprocess
from moviepy.editor import VideoFileClip, concatenate_videoclips

In [2]:
ideas_file='file.txt'
lang="en"
with open(ideas_file,'r') as file:
  ideas=file.readlines()
c=0
for i in range(len(ideas)):
  ideas[i]=ideas[i][0:len(ideas[i])-1]
  c+=len(ideas[i])
print(ideas)
print(c)

['Fondée en 1933,', 'contenant 260 milliards de barils', 'devant les autres société']
73


In [3]:
image_paths = ["image1.jpeg", "image2.jpg", "image3.jpg", "image4.jpeg","image5.jpeg"]
fps=24
total_duration = 0.5*c+0.5
output_path = "video.mp4"
final_video="final.mp4"
print(total_duration,' in seconds')
print(total_duration/60,' in minutes')

line_percentage=[0 for i in ideas]
for i in range(len(ideas)) :
  line_percentage[i]=(total_duration*len(ideas[i])/c)+0.5

print(line_percentage)

37.0  in seconds
0.6166666666666667  in minutes
[8.102739726027398, 17.226027397260275, 13.17123287671233]


#Animations

In [4]:
def zoom_in(image_path, output_path, duration=3, fps=30, max_scale=1.15):
    # Load the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Calculate the scale increment per frame
    scale_increment = (max_scale - 1.0) / (duration * fps)

    # Calculate the center of the image
    center_x, center_y = width // 2, height // 2

    # Create the zoom-out frames
    current_scale = 1.0
    for i in range(int(duration * fps)):
        # Calculate the current scale for this frame
        current_scale += scale_increment

        # Calculate the translation to keep the center fixed
        translation_x = center_x * (1 - current_scale)
        translation_y = center_y * (1 - current_scale)

        # Create a scaling and translation matrix to zoom out the image from the center
        transformation_matrix = np.float32([[current_scale, 0, translation_x], [0, current_scale, translation_y]])

        # Apply the transformation to the image
        zoomed_in_image = cv2.warpAffine(image, transformation_matrix, (width, height))

        # Write the frame to the video
        out.write(zoomed_in_image)

    # Release the video writer
    out.release()


def zoom_out(image_path, output_path, duration=3, fps=30, max_scale=1.15):
    # Load the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Calculate the scale decrement per frame
    scale_decrement = (max_scale - 1.0) / (duration * fps)

    # Calculate the center of the image
    center_x, center_y = width // 2, height // 2

    # Create the zoom-in frames
    current_scale = max_scale
    for i in range(int(duration * fps)):
        # Calculate the current scale for this frame
        current_scale -= scale_decrement

        # Calculate the translation to keep the center fixed
        translation_x = center_x * (1 - current_scale)
        translation_y = center_y * (1 - current_scale)

        # Create a scaling and translation matrix to zoom in the image from the center
        transformation_matrix = np.float32([[current_scale, 0, translation_x], [0, current_scale, translation_y]])

        # Apply the transformation to the image
        zoomed_out_image = cv2.warpAffine(image, transformation_matrix, (width, height))

        # Write the frame to the video
        out.write(zoomed_out_image)

    # Release the video writer
    out.release()

def translate_left(image_path, output_path, duration=3, fps=30):
    # Load the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Calculate the width of the visible portion (90% of the image width)
    visible_width = int(width * 0.9)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (visible_width, height))

    # Calculate the total translation distance for the entire duration
    total_translation_distance = width - visible_width

    # Calculate the translation distance per second
    translation_per_second = total_translation_distance / duration

    # Create the translated frames
    for i in range(int(duration * fps)):
        # Calculate the translation distance for this frame
        translation_distance = int(i / fps * translation_per_second)

        # Create a translation matrix to shift the image to the left
        translation_matrix = np.float32([[1, 0, -translation_distance], [0, 1, 0]])

        # Apply the translation to the image
        translated_image = cv2.warpAffine(image, translation_matrix, (visible_width, height))

        # Write the frame to the video
        out.write(translated_image)

    # Release everything if job is finished
    out.release()


def translate_right(image_path, output_path, duration=3, fps=30):
    # Load the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Calculate the width of the visible portion (90% of the image width)
    visible_width = int(width * 0.9)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (visible_width, height))

    # Calculate the total translation distance for the entire duration
    total_translation_distance = width - visible_width

    # Calculate the translation distance per second
    translation_per_second = total_translation_distance / duration

    # Create the translated frames
    frames = []
    for i in range(int(duration * fps)):
        # Calculate the translation distance for this frame
        translation_distance = int(i / fps * translation_per_second)

        # Create a translation matrix to shift the image to the left
        translation_matrix = np.float32([[1, 0, -translation_distance], [0, 1, 0]])

        # Apply the translation to the image
        translated_image = cv2.warpAffine(image, translation_matrix, (visible_width, height))

        # Append the frame to the frames list
        frames.append(translated_image)

    # Append the frames in reverse order to the output video to get the inverse effect
    for frame in reversed(frames):
        out.write(frame)

    # Release everything if job is finished
    out.release()

#video creation phase 1:

In [5]:
# Function to concatenate video clips into a single video using moviepy
def concatenate_videos(video_paths, output_path, target_resolution=(1280,720)):
    # Create VideoFileClip objects for each video and resize to the target resolution
    video_clips = [VideoFileClip(video_path).resize(target_resolution) for video_path in video_paths]

    # Concatenate the video clips
    final_clip = concatenate_videoclips(video_clips, method="compose")

    # Write the final video to the output path
    final_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")

    # Close the video clips
    for video_clip in video_clips:
        video_clip.close()

# Function to create a video with transitions between images
def create_transition_video(image_paths, output_path, total_duration, fps=30, animations=None):
    if not animations:
        animations = [zoom_in, translate_left, zoom_out, translate_right]

    # Create temporary video animations for each image
    video_paths = []
    i=0
    for image_path in image_paths:
      video_path=f'temp_video_for_image_n_{i}.mp4'
      create_image_animation=random.choice(animations)
      create_image_animation(image_path, video_path, total_duration / len(image_paths), fps)
      video_paths.append(video_path)
      i+=1

    # Concatenate the temporary videos to create the final video
    concatenate_videos(video_paths, output_path)

    # Remove temporary video files
    for video_path in video_paths:
        os.remove(video_path)

In [6]:
create_transition_video(image_paths, output_path, total_duration, fps)

Moviepy - Building video video.mp4.
Moviepy - Writing video video.mp4



t:  20%|█▉        | 177/886 [00:18<01:53,  6.24it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file temp_video_for_image_n_0.mp4, 7464960 bytes wanted but 0 bytes read,at frame 177/178, at time 7.38/7.38 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready video.mp4


#Video creation phase 2:

In [7]:
import cv2
import numpy as np

def add_light_color_effect(arr, alpha=0.5, beta=0.5):
    # Create a yellow tint image
    yellow_tint = np.zeros_like(arr)
    yellow_tint[:, :, 0] = 0  # Hue: 30 (Yellow)
    yellow_tint[:, :, 1] = 0  # Saturation: 100 (Medium Saturation)
    yellow_tint[:, :, 2] = 3  # Value: 255 (Maximum Brightness)

    # Combine the original frame with the yellow tint using alpha blending
    return cv2.addWeighted(arr, alpha, yellow_tint, beta, 0)

def gaussian_blur(arr, sigma):
    return cv2.GaussianBlur(arr, (0, 0), sigma)

def add_blurred_split_screen_effect(input_video_path, output_video_path, padding_percentage=0.05, blur_sigma=5):
    cap = cv2.VideoCapture(input_video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    left_padding = int(width * padding_percentage)
    right_padding = int(width * padding_percentage)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        left_blurred = frame[:, :left_padding].copy()
        left_blurred = gaussian_blur(left_blurred, sigma=blur_sigma)
        left_blurred = add_light_color_effect(left_blurred)

        right_blurred = frame[:, -right_padding:].copy()
        right_blurred = gaussian_blur(right_blurred, sigma=blur_sigma)
        right_blurred = add_light_color_effect(right_blurred)

        combined_frame = frame.copy()
        combined_frame[:, :left_padding] = left_blurred
        combined_frame[:, -right_padding:] = right_blurred

        out.write(combined_frame)

    cap.release()

    # Release the VideoWriter object after writing all frames
    out.release()
    cv2.destroyAllWindows()

# Example usage:
video_path = output_path
output_path = "output.mp4"
add_blurred_split_screen_effect(video_path, output_path, padding_percentage=0.05, blur_sigma=5)

#Video creation phase 3:

##CV2

In [ ]:
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image

# Load the video file
cap = cv2.VideoCapture(output_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Calculate the start and end frames for each subtitle
start_frames = [int(sum(line_percentage[:i]) * fps) for i in range(len(line_percentage))]
end_frames = [int(sum(line_percentage[:i+1]) * fps) for i in range(len(line_percentage))]

# Set up the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(final_video, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

# Add the subtitles to the video
frame_num = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame from BGR to RGB color space
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Convert the frame to a PIL Image
    pil_image = Image.fromarray(frame)

    # Create a drawing context for the PIL Image
    draw = ImageDraw.Draw(pil_image)

    # Check if we need to add a subtitle to this frame
    for i, (start, end) in enumerate(zip(start_frames, end_frames)):
        if start <= frame_num < end:
            # Add the subtitle to the frame
            font_path = '/content/ReadexPro-Bold.ttf'
            font_size = 48
            font = ImageFont.truetype(font_path, font_size)
            text = ideas[i]
            textsize = font.getsize_multiline(text)

            # Position the subtitle on the left side of the frame with a small margin
            textX = 100
            textY = frame.shape[0] - 150 - textsize[1]

            # Add some padding between the text and its black background
            padding = 10

            # Add a black background for the subtitle
            draw.rectangle((textX - padding, textY + textsize[1] + padding, textX + textsize[0] + padding, textY - padding), fill=(0, 0, 0))

            # Put the subtitle text in color #FFC800
            draw.text((textX, textY), text, fill=(255, 200, 0), font=font)

    # Convert the PIL Image back to a NumPy array
    frame = np.array(pil_image)

    # Convert the frame back to BGR color space
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # Write the frame to the output video
    out.write(frame)
    frame_num += 1

# Release resources
cap.release()
out.release()

##modifying the code so the subtitles work for arabic language as well

In [13]:
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image

# Load the video file
cap = cv2.VideoCapture(output_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Calculate the start and end frames for each subtitle
start_frames = [int(sum(line_percentage[:i]) * fps) for i in range(len(line_percentage))]
end_frames = [int(sum(line_percentage[:i+1]) * fps) for i in range(len(line_percentage))]

# Set up the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(final_video, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

# Add the subtitles to the video
frame_num = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame from BGR to RGB color space
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Convert the frame to a PIL Image
    pil_image = Image.fromarray(frame)

    # Create a drawing context for the PIL Image
    draw = ImageDraw.Draw(pil_image)

    # Check if we need to add a subtitle to this frame
    for i, (start, end) in enumerate(zip(start_frames, end_frames)):
        if start <= frame_num < end:
            # Add the subtitle to the frame
            font_path = '/content/ReadexPro-Bold.ttf'
            font_size = 48
            font = ImageFont.truetype(font_path, font_size)
            text = ideas[i]
            textsize = font.getsize_multiline(text)

            # Position the subtitle on the left or right side of the frame with a small margin depending on lang variable
            if lang == 'ar':
                textX = frame.shape[1] - 100 - textsize[0]
            else:
                textX = 100
            textY = frame.shape[0] - 150 - textsize[1]

            # Add some padding between the text and its black background
            padding = 10

            # Add a black background for the subtitle
            draw.rectangle((textX - padding, textY + textsize[1] + padding, textX + textsize[0] + padding, textY - padding), fill=(0, 0, 0))

            # Put the subtitle text in color #FFC800
            draw.text((textX, textY), text, fill=(255, 200, 0), font=font)

    # Convert the PIL Image back to a NumPy array
    frame = np.array(pil_image)

    # Convert the frame back to BGR color space
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # Write the frame to the output video
    out.write(frame)
    frame_num += 1

# Release resources
cap.release()
out.release()


  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize 

##Adding audio

In [9]:
!pip install pyttsx3

In [11]:
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image
import pyttsx3
import time

# Initialize the pyttsx3 engine
engine = pyttsx3.init()

# Load the video file
cap = cv2.VideoCapture(output_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Calculate the start and end frames for each subtitle
start_frames = [int(sum(line_percentage[:i]) * fps) for i in range(len(line_percentage))]
end_frames = [int(sum(line_percentage[:i+1]) * fps) for i in range(len(line_percentage))]

# Set up the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(final_video, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

# Add the subtitles to the video
frame_num = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame from BGR to RGB color space
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Convert the frame to a PIL Image
    pil_image = Image.fromarray(frame)

    # Create a drawing context for the PIL Image
    draw = ImageDraw.Draw(pil_image)

    # Check if we need to add a subtitle to this frame
    for i, (start, end) in enumerate(zip(start_frames, end_frames)):
        if start <= frame_num < end:
            # Add the subtitle to the frame
            font_path = '/content/ReadexPro-Bold.ttf'
            font_size = 48
            font = ImageFont.truetype(font_path, font_size)
            text = ideas[i]
            textsize = font.getsize_multiline(text)

            # Position the subtitle on the left or right side of the frame with a small margin depending on lang variable
            if lang == 'ar':
                textX = frame.shape[1] - 100 - textsize[0]
            else:
                textX = 100
            textY = frame.shape[0] - 150 - textsize[1]

            # Add some padding between the text and its black background
            padding = 10

            # Add a black background for the subtitle
            draw.rectangle((textX - padding, textY + textsize[1] + padding, textX + textsize[0] + padding, textY - padding), fill=(0, 0, 0))

            # Put the subtitle text in color #FFC800
            draw.text((textX, textY), text, fill=(255, 200, 0), font=font)

            # Add a voiceover that reads the subtitle with a 0.5-second delay
            if frame_num == start:
                engine.say(text)
                engine.runAndWait()
                time.sleep(0.5)

    # Convert the PIL Image back to a NumPy array
    frame = np.array(pil_image)

    # Convert the frame back to BGR color space
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # Write the frame to the output video
    out.write(frame)
    frame_num += 1

# Release resources
cap.release()
out.release()


OSError: ignored

#the next cell adds the Logo

In [15]:
from moviepy.editor import VideoFileClip, ImageClip, CompositeVideoClip

def add_logo_to_video(video_path, logo_path, output_path, logo_scale=0.1, padding_left=20):
    # Load the video clip
    video_clip = VideoFileClip(video_path)

    # Load the logo image and resize it
    logo = ImageClip(logo_path)
    logo = logo.resize(height=int(video_clip.h * logo_scale))

    # Set the duration of the logo image to match the duration of the video
    logo = logo.set_duration(video_clip.duration)

    # Calculate the position of the logo to create left padding
    logo_x_pos = padding_left
    logo_y_pos = "top"

    # Set the position of the logo with padding on the left
    logo = logo.set_position((logo_x_pos, logo_y_pos))

    # Composite the logo on top of the video
    final_clip = CompositeVideoClip([video_clip, logo])

    # Write the final video with the logo to a file
    final_clip.write_videofile(output_path, codec='libx264', audio_codec='aac')

    # Close the video clip and free up resources
    final_clip.close()
    video_clip.close()


video_path = "final.mp4"  # Replace with the path to your video
logo_path = "logo.png"    # Replace with the path to your logo image
output_path = "final_video.mp4"  # Replace with the desired output path

add_logo_to_video(video_path, logo_path, output_path)


Moviepy - Building video final_video.mp4.
Moviepy - Writing video final_video.mp4



t: 100%|██████████| 887/887 [01:15<00:00, 14.21it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file final.mp4, 2764800 bytes wanted but 0 bytes read,at frame 886/887, at time 36.92/36.92 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready final_video.mp4
Moviepy - Building video final_video.mp4.
Moviepy - Writing video final_video.mp4



Moviepy - Done !
Moviepy - video ready final_video.mp4
